In [18]:
import numpy as np
import pandas as pd
import os 
import sklearn
import matplotlib.pyplot as plt
from sklearn import  linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from fancyimpute import KNN, NuclearNormMinimization, SoftImpute, IterativeImputer, BiScaler

In [3]:
df = pd.read_csv('qudditch_training.csv')

In [4]:
df.shape

(101266, 48)

In [5]:
# Replace gender with integer male = 0 female = 1 
s = df['gender']
s.replace('Female',1,inplace=True)
s.replace('Male',0,inplace=True)


In [6]:
df = df[df.gender != 'Unknown/Invalid']
df = df[df.house != '?']
#drop the useless columns 



#     print(df[i])
df['weight'].replace({'>200': 200, '[0-25)': 0,'[25-50)': 25,'[50-75)': 50,'[75-100)': 75,'[100-125)': 100,'[125-150)': 120,'[150-175)': 150,'[175-200)': 175,'?':np.NaN},inplace=True)



#### since df['transylvanian_tackle']) only has two value and the 'steady' has only 2 rows. so we could remove it.

In [7]:
# Finding columns with no information
useless = []
for i in df.columns:
    l = len(df[i].unique())
    if l<2:
        useless.append(i)
        print("Column Name: ",i,"Uniques: ",l)
df.drop(useless,axis=1,inplace=True)        

Column Name:  double_eight_loop Uniques:  1
Column Name:  finbourgh_flick Uniques:  1
Column Name:  transylvanian_tackle Uniques:  1


In [9]:
bi = []
for i in df.columns:
    l = len(df[i].unique())
    if l == 2:
        bi.append(i)
        print("Column Name: ",i,"Uniques: ",l)
        print(df[i].unique())

#change all feature that only has two items into 0,1 which all no indicated by 0 and other indicated by 1 
for i in bi:
    df[i].replace(df[i].unique()[0],0,inplace=True)
    df[i].replace(df[i].unique()[1],1,inplace=True)        



Column Name:  power_play Uniques:  2
['No' 'Steady']
Column Name:  starfish_and_stick Uniques:  2
['No' 'Steady']
Column Name:  chelmondiston_charge Uniques:  2
['No' 'Steady']
Column Name:  plumpton_pass Uniques:  2
['No' 'Steady']
Column Name:  porskoff_ploy Uniques:  2
['No' 'Steady']
Column Name:  woollongong_shimmy Uniques:  2
['No' 'Steady']
Column Name:  change Uniques:  2
['No' 'Ch']
Column Name:  snitch_caught Uniques:  2
['No' 'Yes']
Column Name:  quidditch_league_player Uniques:  2
['NO' 'YES']


In [10]:
#analysis of correlation of all numerical featrue 
cor = {}
for name in df.columns:
    if df[name].dtype == 'int64' or df[name].dtype == 'float64':
        corr = df[name].corr(df['quidditch_league_player'])
    else:
        continue
    cor[name] = corr
sor = sorted(cor.items(),key=lambda x:x[1])
#may find that some of id don't have much correlation with label
sor

[('id_num', -0.008663130280845154),
 ('player_id', 0.008518181042934395),
 ('gender', nan),
 ('weight', -0.02560953316613464),
 ('foul_type_id', -0.011746223119824695),
 ('num_game_losses', -0.011482509382814463),
 ('starfish_and_stick', -0.0031532090628339695),
 ('chelmondiston_charge', -0.001957066408769401),
 ('plumpton_pass', -0.0012812077185507325),
 ('porskoff_ploy', -0.0011299014047218398),
 ('woollongong_shimmy', -0.001129901404721839),
 ('power_play', -0.0011299014047218363),
 ('penalty_id', 0.005173892502892625),
 ('age', 0.01674118745233656),
 ('num_games_satout', 0.018810928811069067),
 ('change', 0.01986573374580235),
 ('num_game_moves', 0.020528687676967753),
 ('snitch_caught', 0.02801700772280525),
 ('num_practice_sessions', 0.03944165223470479),
 ('game_duration', 0.04433395997197832),
 ('num_games_won', 0.049653484643935),
 ('game_move_id', 0.05053896966396047),
 ('num_games_injured', 0.061159644772269664),
 ('num_games_notpartof', 0.16547799643760738),
 ('quidditch_le

# Remove some non-category feature and dummie remain features

In [96]:
cat = []
for i in df.columns:
    l = len(df[i].unique())
    if l >= 2:
        cat.append(i)
#         print("Column Name: ",i,"Uniques: ",l)
#         print(df[i].unique())
cat.remove('id_num')
cat.remove('player_id')
cat.remove('weight')
cat.remove('player_code')
cat.remove('move_specialty')
# cat.remove('weight')
# cat.remove('player_code')
# cat.remove('move_specialty')

dummie = pd.get_dummies(df[cat])

idd = df['id_num']  
dummie = pd.concat([dummie,idd],axis=1)
dummie

# seperately add feature on the set to imputation

In [99]:
wei = df['weight']
player = df['player_code']
temp_weight = pd.concat([dummie,wei],axis=1)
temp_play = pd.concat([dummie,player],axis=1)
temp_play.replace('?',np.NaN,inplace=True)


In [100]:
# split data for weight

# X_filled_nnm = NuclearNormMinimization().fit_transform(temp_weight)
weight_null = temp_weight.loc[temp_weight['weight'].isnull()]
weight_no_null = temp_weight.dropna()
wei_train_x = weight_no_null.iloc[:,:-1]
wei_train_y = weight_no_null.iloc[:,-1:]
wei_test_x = weight_null.iloc[:,:-1]

In [101]:
# Create linear regression object
regr = linear_model.LinearRegression()
# Train the model using the training sets
regr.fit(wei_train_x, wei_train_y)
# Make predictions using the testing set
wei_test_y = regr.predict(wei_test_x)

In [102]:
# The coefficients
print('Coefficients: \n', regr.coef_)



Coefficients: 
 [[-6.26816492e+00 -5.63036296e-01 -1.06873039e-01 -1.25040118e-01
  -2.19055135e-01 -5.77504586e-02  3.40408141e-01  1.25730542e-01
   1.23258250e-01 -1.30546497e+00 -1.23975200e+00  1.79121844e+00
   7.61181568e-15  1.08472847e+01 -6.35481188e-15  1.35681823e-14
   2.32565844e-14  1.69717117e-14 -4.24780931e-02 -1.81159906e+00
  -1.95903759e+00  8.50895961e+00  2.97578373e-15  7.38567691e-01
  -1.66587584e+01  7.41123108e+00 -1.09593819e+01 -1.11683592e+00
  -8.13939606e-01  1.34246040e+00  1.73217256e+00  1.52618516e+00
   5.41243058e+00  2.89252153e+00 -1.56128044e-02  3.42061533e-15
   2.43254695e-15 -4.52535241e-15 -9.22711498e-15  2.59867109e+00
  -3.19250909e+00 -1.51774199e+00  2.11157999e+00  6.73711012e+00
  -4.99903676e+00 -1.46739319e+00 -2.70680178e-01 -1.69455918e-14
  -1.57596670e+00  2.17981744e+00 -6.03850737e-01 -1.87646018e-15
  -4.86827192e-01 -1.69595552e-01  6.56422744e-01 -4.00741094e-15
  -1.53588758e+01  1.53588758e+01 -4.29992427e-15 -8.1201285

In [129]:
#concat the result to original dataset 
df_test_y = pd.DataFrame(wei_test_y)
df_test_y.reset_index(drop=True, inplace=True)
wei_test_x.reset_index(drop=True, inplace=True)
result = pd.concat([wei_test_x,df_test_y],axis=1)
result.rename(index=str, columns={0: "weight"},inplace=True)
result=result.append(weight_no_null)

In [ ]:
cor_weight = {}

for name in no_ques.columns:
    if no_ques[name].dtype == 'int64' or no_ques[name].dtype == 'float64':
        corr = no_ques[name].corr(no_ques['weight'])
    else:
        continue
    cor_weight[name] = corr
sor_weight = sorted(cor_weight.items(),key=lambda x:x[1])
sor_weight

In [130]:
#add play_code into set
id_ply = df[['id_num','player_code']]

result = result.join(id_ply.set_index('id_num'),on='id_num')

In [137]:
#sort index
result.sort_index(by='id_num')
result.set_index('id_num',inplace=True)

/Users/nightmare/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: by argument to sort_index is deprecated, please use .sort_values(by=...)
  """Entry point for launching an IPython kernel.


In [142]:
#split set by player_code
result['player_code'].replace('?',np.NaN,inplace=True)

code_null = result.loc[result['player_code'].isnull()]
code_no_null = result.dropna()
code_train_x = code_no_null.iloc[:,:-1]
code_train_y = code_no_null.iloc[:,-1:]
code_test_x = code_null.iloc[:,:-1]

In [241]:
code_train_y['player_code'].unique()

array(['MC', 'MD', 'HM', 'UN', 'BC', 'SP', 'CP', 'SI', 'DM', 'CM', 'CH',
       'PO', 'WC', 'OT', 'OG', 'MP'], dtype=object)

In [244]:
#deal with the label using label binarizer
from sklearn.preprocessing import LabelBinarizer
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(code_train_y['player_code'])
trans=le.transform(code_train_y['player_code']) 


In [247]:
#bayes 
bys = linear_model.BayesianRidge()
bys.fit(code_train_x,trans)
pre = bys.predict(code_test_x)

In [248]:
#process and concat
pre=np.around(pre)
pre = pre.astype('int64')

pre_y=le.inverse_transform(pre)

code_test_x['player_code'] = pre_y

result=code_no_null.append(code_test_x)

result.sort_index(by='id_num')

In [297]:

id_spc = df[['id_num','move_specialty']]
# result = result.join(id_spc,on='id_num')
# result['move_specialty'].replace('?',np.NaN,inplace=True)

spc_null = result.loc[result['move_specialty'].isnull()]
spc_no_null = result.dropna()
spc_train_x = spc_no_null.iloc[:,:-2]
spc_train_y = spc_no_null.iloc[:,-1:]
spc_test_x = spc_null.iloc[:,:-2]


In [358]:
le1 = preprocessing.LabelEncoder()
le1.fit(spc_train_y['move_specialty'])

LabelEncoder()

In [359]:
trans_spc=le1.transform(spc_train_y['move_specialty'])

In [366]:
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                    hidden_layer_sizes=(5, 2), random_state=1,max_iter=3000,learning_rate_init=1)

clf.fit(spc_train_x, trans_spc)      

pre_spc_nn = clf.predict_proba(spc_test_x)

unique, counts = np.unique(pre_spc_nn, return_counts=True)
dict(zip(unique, counts))

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(5, 2), learning_rate='constant',
       learning_rate_init=1, max_iter=3000, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=1, shuffle=True, solver='lbfgs', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [374]:
from sklearn import neighbors
n_neighbors = 15
clf_knn = neighbors.KNeighborsClassifier(n_neighbors, weights="distance")
clf_knn.fit(spc_train_x, trans_spc)
pre_knn=clf_knn.predict(spc_test_x)

In [378]:
pre_knn_in=le1.inverse_transform(pre_knn)

unique, counts = np.unique(pre_knn_in, return_counts=True)
dict(zip(unique, counts))

{'specialty12': 6038,
 'specialty13': 67,
 'specialty14': 2,
 'specialty15': 1,
 'specialty16': 10,
 'specialty19': 21570,
 'specialty20': 219,
 'specialty21': 25,
 'specialty25': 379,
 'specialty26': 13,
 'specialty27': 2,
 'specialty28': 1487,
 'specialty29': 419,
 'specialty31': 23,
 'specialty34': 11,
 'specialty36': 7,
 'specialty38': 34,
 'specialty4': 3299,
 'specialty44': 221,
 'specialty46': 6,
 'specialty48': 131,
 'specialty51': 2,
 'specialty52': 28,
 'specialty53': 625,
 'specialty60': 25,
 'specialty61': 274,
 'specialty63': 1177,
 'specialty65': 94,
 'specialty69': 13,
 'specialty70': 95,
 'specialty71': 1,
 'specialty72': 165,
 'specialty9': 12057}

In [391]:
spc_test_x['move_specialty'] = pre_knn_in
spc_test_x['player_code'] = spc_null['player_code']
result=spc_no_null.append(spc_test_x)
result.sort_index(by='id_num')

/Users/nightmare/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: by argument to sort_index is deprecated, please use .sort_values(by=...)
  after removing the cwd from sys.path.


,age,bludger_backbeat_Down,bludger_backbeat_No,bludger_backbeat_Steady,bludger_backbeat_Up,body_blow_Down,body_blow_No,body_blow_Steady,body_blow_Up,change,...,weight,woollongong_shimmy,wronski_feint_Down,wronski_feint_No,wronski_feint_Steady,wronski_feint_Up,zig-zag_Down,zig-zag_No,zig-zag_Steady,zig-zag_Up
id_num,,,,,,,,,,,,,,,,,,,,,
1,11.0,0,1,0,0,0,1,0,0,0,...,69.132447,0,0,1,0,0,0,1,0,0
2,12.0,0,1,0,0,0,1,0,0,1,...,96.018952,0,0,1,0,0,0,1,0,0
3,13.0,0,1,0,0,0,1,0,0,0,...,85.540381,0,0,1,0,0,0,1,0,0
4,14.0,0,1,0,0,0,1,0,0,1,...,81.074588,0,0,1,0,0,0,1,0,0
5,14.5,0,1,0,0,0,1,0,0,1,...,69.734277,0,0,1,0,0,0,1,0,0
6,15.0,0,1,0,0,0,1,0,0,0,...,79.106847,0,0,1,0,0,0,1,0,0
7,15.5,0,1,0,0,0,0,1,0,1,...,78.898584,0,0,1,0,0,0,1,0,0
8,16.0,0,1,0,0,0,1,0,0,0,...,66.563418,0,0,1,0,0,0,1,0,0
9,16.5,0,1,0,0,0,1,0,0,1,...,64.125383,0,0,1,0,0,0,1,0,0


In [395]:
#dummie again
ply_and_spc = ['player_code','move_specialty']
dummie_ag = pd.get_dummies(result[ply_and_spc])
result = pd.concat([result,dummie_ag],axis=1)

In [398]:
result=result.sort_index()

In [318]:
# bys_spc = linear_model.BayesianRidge()
# bys_spc.fit(spc_train_x,trans_spc)
# pre_spc = bys_spc.predict(spc_test_x)

# pre_spc=np.around(pre_spc)
# pre_spc = pre_spc.astype('int64')

# unique, counts = np.unique(spc_train_y, return_counts=True)
# dict(zip(unique, counts))